In [29]:
import motrainer
import xarray as xr

## Load data

In [30]:
ds = xr.open_zarr('../example/example1_data.zarr/')

In [31]:
ds

<xarray.Dataset>
Dimensions:    (space: 5, time: 8506)
Coordinates:
    latitude   (space) float64 dask.array<chunksize=(5,), meta=np.ndarray>
    longitude  (space) float64 dask.array<chunksize=(5,), meta=np.ndarray>
  * time       (time) datetime64[ns] 2007-01-02 ... 2020-01-01T01:00:00
Dimensions without coordinates: space
Data variables:
    BIOMA1     (space, time) float64 dask.array<chunksize=(3, 8506), meta=np.ndarray>
    BIOMA2     (space, time) float64 dask.array<chunksize=(3, 8506), meta=np.ndarray>
    TG1        (space, time) float64 dask.array<chunksize=(3, 8506), meta=np.ndarray>
    TG2        (space, time) float64 dask.array<chunksize=(3, 8506), meta=np.ndarray>
    TG3        (space, time) float64 dask.array<chunksize=(3, 8506), meta=np.ndarray>
    WG1        (space, time) float64 dask.array<chunksize=(3, 8506), meta=np.ndarray>
    WG2        (space, time) float64 dask.array<chunksize=(3, 8506), meta=np.ndarray>
    WG3        (space, time) float64 dask.array<chunksize=(3, 8506), meta=np.ndarray>
    curv       (space, time) float64 dask.array<chunksize=(3, 8506), meta=np.ndarray>
    sig        (space, time) float64 dask.array<chunksize=(3, 8506), meta=np.ndarray>
    slop       (space, time) float64 dask.array<chunksize=(3, 8506), meta=np.ndarray>
Attributes:
    license:  data license
    source:   data source

## Split per gridcell

In [36]:
# Split in each space gridcell
ds = ds.assign_coords(space = (("space"), range(ds.space.shape[0])))
ds

<xarray.Dataset>
Dimensions:    (space: 5, time: 8506)
Coordinates:
    latitude   (space) float64 dask.array<chunksize=(5,), meta=np.ndarray>
    longitude  (space) float64 dask.array<chunksize=(5,), meta=np.ndarray>
  * time       (time) datetime64[ns] 2007-01-02 ... 2020-01-01T01:00:00
  * space      (space) int64 0 1 2 3 4
Data variables:
    BIOMA1     (space, time) float64 dask.array<chunksize=(3, 8506), meta=np.ndarray>
    BIOMA2     (space, time) float64 dask.array<chunksize=(3, 8506), meta=np.ndarray>
    TG1        (space, time) float64 dask.array<chunksize=(3, 8506), meta=np.ndarray>
    TG2        (space, time) float64 dask.array<chunksize=(3, 8506), meta=np.ndarray>
    TG3        (space, time) float64 dask.array<chunksize=(3, 8506), meta=np.ndarray>
    WG1        (space, time) float64 dask.array<chunksize=(3, 8506), meta=np.ndarray>
    WG2        (space, time) float64 dask.array<chunksize=(3, 8506), meta=np.ndarray>
    WG3        (space, time) float64 dask.array<chunksize=(3, 8506), meta=np.ndarray>
    curv       (space, time) float64 dask.array<chunksize=(3, 8506), meta=np.ndarray>
    sig        (space, time) float64 dask.array<chunksize=(3, 8506), meta=np.ndarray>
    slop       (space, time) float64 dask.array<chunksize=(3, 8506), meta=np.ndarray>
Attributes:
    license:  data license
    source:   data source

In [37]:
bags = ds.mot.dataset_split('space')
bags

dask.bag<from_sequence, npartitions=5>

In [43]:
bags.compute()

[<xarray.Dataset>
 Dimensions:    (space: 1, time: 8506)
 Coordinates:
     latitude   (space) float64 dask.array<chunksize=(1,), meta=np.ndarray>
     longitude  (space) float64 dask.array<chunksize=(1,), meta=np.ndarray>
   * time       (time) datetime64[ns] 2007-01-02 ... 2020-01-01T01:00:00
   * space      (space) int64 0
     space_id   (space) int64 0
 Data variables:
     BIOMA1     (space, time) float64 dask.array<chunksize=(1, 8506), meta=np.ndarray>
     BIOMA2     (space, time) float64 dask.array<chunksize=(1, 8506), meta=np.ndarray>
     TG1        (space, time) float64 dask.array<chunksize=(1, 8506), meta=np.ndarray>
     TG2        (space, time) float64 dask.array<chunksize=(1, 8506), meta=np.ndarray>
     TG3        (space, time) float64 dask.array<chunksize=(1, 8506), meta=np.ndarray>
     WG1        (space, time) float64 dask.array<chunksize=(1, 8506), meta=np.ndarray>
     WG2        (space, time) float64 dask.array<chunksize=(1, 8506), meta=np.ndarray>
     WG3      

## Train Test Split

In [44]:
import dask_ml.model_selection as dcv

def to_dataframe(ds):
    return ds.to_dask_dataframe()

def chunk(ds, chunks):
    return ds.chunk(chunks)

test_size=0.33
f_shuffle=True

bags = bags.map(chunk, {'space':100}).map(to_dataframe)
train_test_bags = bags.map(dcv.train_test_split, test_size=test_size, shuffle=f_shuffle, random_state=1) # train_test_split only takes da, df or series. No ds.

In [49]:
bags = train_test_bags.compute()

0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.


In [52]:
train_bag = train_test_bags.pluck(0)